In [1]:
import torch
import pandas as pd
import numpy as np
# 新数据
# 读取所有数据
paths = [
    '../data/clean/clean_processed_data_1500_1.csv', '../data/clean/clean_processed_data_1500_2.csv', '../data/clean/clean_processed_data_1500_3.csv',
    '../data/clean/clean_processed_data_1500_4.csv', '../data/clean/clean_processed_data_1500_5.csv', '../data/clean/clean_processed_data_1500_6.csv'
]

# 读取并拼接数据
# all_data = pd.concat([pd.read_csv(path) for path in paths], ignore_index=True)
# all_data = np.array(all_data)
# all_data.shape

In [4]:
all_data = []
for path in paths:
    data = pd.read_csv(path)
    data = np.array(data)
    all_data.append(data)
all_data = np.concatenate(all_data, axis=0)

In [ ]:
all_data.shape

In [5]:
# 
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy
import pandas as pd
# 设定随机种子以保证结果可复现
torch.manual_seed(0)
np.random.seed(0)

# 设定设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# 处理数据的函数
def process_data(data):
    pro_data = []
    for record in data:
        # 每条记录是一个字符串，将其按分号分割
        elements = record[0].split(';')
        processed_data = []
        for element in elements:
            # print(element)
            # 去除括号并按逗号分割成元素和数字
            mol, count = element.strip('()').split(',')
            processed_data.append((mol, int(count)))
        pro_data.append(np.array(processed_data))
    return pro_data

# 调用函数处理数据
processed = process_data(all_data) 
processed = np.array(processed, dtype=object)
print(processed.shape)

(2652,)


# 对原始数据(processed)继续进行统计分析

In [7]:
# 统计所有物种类型
unique_species = set()

for record in processed:
    for element in record:
        # element 是一个包含 ['物种名称', '停留时间'] 的数组
        mol = element[0]  # 获取物种名称
        unique_species.add(mol)

# 输出所有物种类型的名称
print(len(unique_species))
#print(unique_species)


39


In [8]:
from collections import Counter
import numpy as np
import pandas as pd

def count_species_with_time(processed, n):
    """
    统计每个物种的总出现次数以及停留时间小于等于n的次数。

    参数:
    - processed (np.array): 处理后的数据数组，元素为每条记录的物种及停留时间。
    - n (int): 停留时间的阈值。

    返回:
    - species_counter (Counter): 每个物种的总出现次数。
    - species_counter_le_n (Counter): 每个物种停留时间小于等于n的出现次数。
    """
    species_counter = Counter()
    species_counter_le_n = Counter()
    
    for record in processed:
        for element in record:
            mol = element[0]
            try:
                count = int(element[1])
            except ValueError:
                print(f"无法将停留时间转换为整数: {element[1]}，跳过该元素。")
                continue  # 跳过无法转换的元素
            
            species_counter[mol] += 1
            if count <= n:
                species_counter_le_n[mol] += 1
    
    return species_counter, species_counter_le_n

def print_species_statistics(species_counts, species_counts_le_n, n):
    """
    打印每种物种的总出现次数和停留时间小于等于n的出现次数，按总出现次数降序排列。

    参数:
    - species_counts (Counter): 每个物种的总出现次数。
    - species_counts_le_n (Counter): 每个物种停留时间小于等于n的出现次数。
    - n (int): 停留时间的阈值。
    """
    # 获取按总出现次数降序排列的物种列表
    sorted_species = species_counts.most_common()
    
    # 打印表头
    print(f"\n{'物种类型':<20} {'总出现次数':<15} {'停留时间<= ' + str(n) + ' 次数':<20}")
    print("-" * 55)
    
    # 打印每种物种的统计数据
    for mol, total in sorted_species:
        le_n = species_counts_le_n.get(mol, 0)
        print(f"{mol:<20} {total:<15} {le_n:<20}")



# 示例调用
if __name__ == "__main__":

    
    n = 5  # 设定阈值
    
    # 统计物种出现次数和停留时间小于等于n的出现次数
    species_counts, species_counts_le_n = count_species_with_time(processed, n)
    
    # 打印统计结果，按总出现次数降序排列
    print_species_statistics(species_counts, species_counts_le_n, n)



物种类型                 总出现次数           停留时间<= 5 次数         
-------------------------------------------------------
MoS5                 63108           47265               
MoS6                 57214           6137                
MoOS6                35233           2670                
MoS4                 33309           22909               
MoS7                 27471           2338                
MoS8                 27364           5718                
MoO2S5               22761           4153                
MoOS5                22418           2624                
MoS9                 16688           7319                
MoOS7                15167           6013                
MoS3                 10009           3507                
MoOS4                8158            3937                
MoS2                 7251            2696                
MoOS2                6446            2368                
MoS                  6235            2451                
MoOS3          

In [9]:
from collections import Counter, defaultdict
# -----------------------------------------------
# 统计每个物种的最大、最小和平均停留时间
# -----------------------------------------------

# 使用 defaultdict 存储每个物种的最小、最大停留时间及总停留时间
species_time_stats = defaultdict(lambda: {'min': None, 'max': None, 'total': 0, 'count': 0})

for record in processed:
    for element in record:
        mol, count = element[0], int(element[1])
        if species_time_stats[mol]['min'] is None or count < species_time_stats[mol]['min']:
            species_time_stats[mol]['min'] = count
        if species_time_stats[mol]['max'] is None or count > species_time_stats[mol]['max']:
            species_time_stats[mol]['max'] = count
        species_time_stats[mol]['total'] += count
        species_time_stats[mol]['count'] += 1

# 计算平均停留时间并准备输出列表
species_time_list = sorted(
    [
        (
            stats['total'] / stats['count'] if stats['count'] > 0 else 0,
            stats['max'],
            stats['min'],
            mol
        )
        for mol, stats in species_time_stats.items()
    ],
    key=lambda x: x[0]
)

# 输出统计结果
print("\n每种物种类型的最大、最小和平均停留时间：")
for avg_time, max_time,min_time,mol in species_time_list:
    print(f"{mol}: 最小停留时间 = {min_time}, 最大停留时间 = {max_time}, 平均停留时间 = {avg_time:.2f}")






每种物种类型的最大、最小和平均停留时间：
Mo2O4: 最小停留时间 = 1, 最大停留时间 = 69, 平均停留时间 = 4.71
Mo2O3: 最小停留时间 = 1, 最大停留时间 = 26, 平均停留时间 = 6.44
MoO3: 最小停留时间 = 1, 最大停留时间 = 473, 平均停留时间 = 8.19
Mo2OS: 最小停留时间 = 1, 最大停留时间 = 43, 平均停留时间 = 9.71
Mo3O9S2: 最小停留时间 = 1, 最大停留时间 = 62, 平均停留时间 = 11.20
Mo2O2: 最小停留时间 = 1, 最大停留时间 = 62, 平均停留时间 = 13.27
MoO2S2: 最小停留时间 = 1, 最大停留时间 = 730, 平均停留时间 = 14.52
MoO2S3: 最小停留时间 = 1, 最大停留时间 = 459, 平均停留时间 = 16.78
MoO2S: 最小停留时间 = 1, 最大停留时间 = 604, 平均停留时间 = 16.86
MoS9: 最小停留时间 = 1, 最大停留时间 = 1877, 平均停留时间 = 20.47
MoO2S4: 最小停留时间 = 1, 最大停留时间 = 1976, 平均停留时间 = 26.09
MoOS7: 最小停留时间 = 1, 最大停留时间 = 1588, 平均停留时间 = 29.18
MoO2S6: 最小停留时间 = 1, 最大停留时间 = 405, 平均停留时间 = 30.17
MoS5: 最小停留时间 = 1, 最大停留时间 = 2017, 平均停留时间 = 36.82
MoOS4: 最小停留时间 = 1, 最大停留时间 = 1124, 平均停留时间 = 37.66
Mo: 最小停留时间 = 1, 最大停留时间 = 544, 平均停留时间 = 38.32
MoO2S5: 最小停留时间 = 1, 最大停留时间 = 1414, 平均停留时间 = 41.01
MoS4: 最小停留时间 = 1, 最大停留时间 = 2915, 平均停留时间 = 41.36
MoOS2: 最小停留时间 = 1, 最大停留时间 = 1242, 平均停留时间 = 42.36
MoO2: 最小停留时间 = 1, 最大停留时间 = 665, 平均停留时间 = 44.41
MoS: 最小停留时间 = 1, 最大停

# 对原始数据(processed)继续进行深度处理

In [21]:
import numpy as np

def filter_and_merge_one_record(record, min_time=5):
    """
    对单条记录进行过滤和合并操作：
    1. 删除停留时间小于 min_time 的状态。
    2. 将删除后连续且相同物种的状态合并为一个状态（停留时间累加）。
    
    参数：
    - record: 形如 array([['MoO','33'], ['MoS','251'], ['MoS','254'], ...]) 的数组
    - min_time: 停留时间的阈值，删除停留时间小于该值的状态
    
    返回：
    - 处理后的 NumPy 数组，满足以上两步操作
    """
    # 1) 过滤：删除停留时间 < min_time 的状态
    filtered = [row for row in record if int(row[1]) >= min_time]
    
    # 2) 合并连续相同物种的状态
    merged = []
    for mol, time_str in filtered:
        time_val = int(time_str)
        if merged and merged[-1][0] == mol:
            # 若与上一个物种相同，则合并（累加停留时间）
            merged[-1][1] = str(int(merged[-1][1]) + time_val)
        else:
            # 否则添加新的物种状态
            merged.append([mol, str(time_val)])
    
    return np.array(merged)

def filter_and_merge_processed(processed, min_time=5):
    """
    对完整的 processed 数据进行过滤和合并操作：
    对 processed 每个元素（单条记录）调用 filter_and_merge_one_record
    
    参数：
    - processed: 形如 [record_1, record_2, ...] 的对象数组（dtype=object）
    - min_time: 停留时间的阈值
    
    返回：
    - 处理后的 NumPy 对象数组
    """
    new_processed = []
    for idx, record in enumerate(processed):
        merged_record = filter_and_merge_one_record(record, min_time=min_time)
        new_processed.append(merged_record)
    
    return np.array(new_processed, dtype=object)

# -------------------- 测试示例 --------------------
if __name__ == "__main__":    
    # 设定阈值，比如 n=5
    n = 50
    
    # 调用过滤和合并函数
    new_processed = filter_and_merge_processed(processed, min_time=n)
    
    # 打印过滤前和过滤后的结果
    # print("过滤前 (原始数据) ：")
    # print(processed[0])
    
    print("\n过滤后 (删除停留时间 < 5 并合并相邻相同状态) ：")
    print(new_processed[0][:20])



过滤后 (删除停留时间 < 5 并合并相邻相同状态) ：
[['Mo3O9' '55']
 ['MoO2' '128']
 ['MoOS2' '75']
 ['MoS4' '366']
 ['MoS6' '196']
 ['MoS4' '156']
 ['MoS6' '1211']
 ['Mo2S7' '52']
 ['MoS6' '1035']
 ['Mo2S6' '329']
 ['MoS4' '51']
 ['MoS5' '61']
 ['MoS4' '191']
 ['MoOS5' '115']
 ['Mo2S10' '420']
 ['MoS6' '127']
 ['MoS7' '2263']
 ['MoS6' '217']
 ['MoS8' '192']
 ['MoS6' '930']]


In [16]:
new_processed[0][0][1]

'55'

In [18]:
for record in new_processed:
    for element in record:
        if int(element[1]) <= 5:
            print(element)

In [22]:
np.save('../data/new_processed50.npy', new_processed)

: 

In [32]:
# 加载 .npy 文件
loaded_data = np.load('../data/new_processed.npy')
# print(loaded_data)

ValueError: Object arrays cannot be loaded when allow_pickle=False

# 数据编码操作

In [ ]:
import numpy as np
# 编码，转换成向量
def incode(molecule):
    vec = np.zeros(3)
    length = len(molecule)
    Mo = molecule.find('Mo')
    O = molecule.find('O')
    S = molecule.find('S')
    # print(Mo, O, S)
    moi = oi = si = 0
    moi = 1
    if O==-1 and S==-1 and length>2:
        moi = molecule[Mo+2:]
    elif O != -1:
        if O > Mo + 2:
            moi = int(molecule[Mo+2:O])
        else:
            moi = 1
        if S == O + 1 or O+1==length:
            oi = 1
        elif S==-1:
            oi = int(molecule[O+1:])
        else:
            oi = int(molecule[O+1:S])
    elif S != -1:
        if S > Mo + 2:
            moi = int(molecule[Mo+2:S])
        else:
            moi = 1
        if S+1==length:
            si = 1
        else:
            si = int(molecule[S+1:])
    if S!=-1:
        if S+1==length:
            si = 1
        else:
            si = int(molecule[S+1:])
    vec = np.array([moi, oi, si])
    return vec
incode('MoOS13')